In [ ]:
from __future__ import division, print_function
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import tensorflow as tf
import math
from tensorflow.python.framework.ops import reset_default_graph
import datetime
import os
import glob
import data_utils

from sklearn.utils import shuffle

## Data loader and Batch generator

In [ ]:
# ==== CAMVID ==== #
images_path = os.path.join('data', 'CamVid')


# loading data and setting up constants
NUM_CLASSES = 11
IMAGE_SHAPE = (480, 360, 3)
data = data_utils.load_data(images_path = images_path)
# to visualize the size of the dimensions of the data
# print
print("@@@Shape checking of data sets@@@")
# print
print(len(data.train))
print("TRAIN")
print("\timages\t%s%f" % (data.train[0].shape, data.train[0].mean()))
# print()
print("VALID")
print("\timages\t%s%f" % (data.valid[0].shape, data.valid[0].mean()))
# print()
print("TEST")
print("\timages\t%s\t%f" % (data.test[0].shape, data.test[0].mean()))


# Batch generation
dummy_batch_gen = data_utils.batch_generator(data, batch_size=10, num_classes=NUM_CLASSES, num_iterations=5e3, seed=42)
train_batch, y_train_batch = next(dummy_batch_gen.gen_train())
_, valid_batch, y_valid_batch = next(dummy_batch_gen.gen_valid())
_, test_batch, y_test_batch = next(dummy_batch_gen.gen_test())

print("TRAIN")
print("\timages,", train_batch.shape)
print()
print("VALID")
print("\timages,", valid_batch.shape)
print()
print("TEST")
print("\timages,", test_batch.shape)

## Model definition

In [ ]:
from tensorflow import layers
from tensorflow.contrib.layers import fully_connected, convolution2d, convolution2d_transpose, batch_norm, max_pool2d, dropout
from tensorflow.python.ops.nn import relu, elu, relu6, sigmoid, tanh, softmax, softplus

# reset graph
reset_default_graph()

# -- THE MODEL --#
num_channels = IMAGE_SHAPE[2] #RGB
num_classes = NUM_CLASSES
k = 16;
height = IMAGE_SHAPE[1]
width = IMAGE_SHAPE[0]
layers_architecture = [4, 5] #Number of layers in denseblocks
layers_bottleneck = 7


# Layer definitions
def layer(x, units):
    with tf.name_scope('layer_' + str(units)):
        x = fully_connected(x, num_outputs=units, activation_fn=relu,
                             normalizer_fn=batch_norm)
        x = convolution2d(x, num_outputs=units, kernel_size=(3, 3),
                                 stride=1)
        return dropout(x, is_training=is_training_pl)
    
def dense_block(x, num_layers):
    with tf.name_scope('dense_' + str(num_layers)):
        for i in range(num_layers):
            layer_output = layer(x, k)
            x = tf.concat([x, layer_output], axis=-1)
            if i == 0:
                res = layer_output
            else:
                res = tf.concat([res, layer_output], axis=-1)
        return res
    

def transition_up(x, units):
    return convolution2d_transpose(x, num_outputs=units, kernel_size=(3, 3), stride=2)
    
    
def transition_down(x, units):
    with tf.name_scope('transition_down_' + str(units)):
        x = batch_norm(x) #Batch norm should be included in fully_connected layer below
        x = relu(x)
        x = convolution2d(x, num_outputs=units, kernel_size=(1, 1),
                             stride=1)
        x = dropout(x, is_training=is_training_pl)
        x = max_pool2d(x, kernel_size=(2, 2))
        return x

# - Tiramisu Architecture - #
# Input placeholder
x_pl = tf.placeholder(tf.float32, [None, height, width, num_channels], 'x_pl')
y_pl = tf.placeholder(tf.float32, [None, height, width, num_classes], 'y_pl')
is_training_pl = tf.placeholder(tf.bool, name="is-training_pl")
print('x_pl', x_pl.shape)
print('y_pl', y_pl.shape)

def upsample(x, skip, num_dense, skip_up=False):
    x = transition_up(x, x.shape[-1].value)
    x = tf.concat([x, skip], axis=-1)
    dense_out = dense_block(x, num_dense)
    if skip_up:
        x = tf.concat([x, dense_out], axis=-1)
    else:
        x = dense_out
    print('DB ({} layers) + TU'.format(num_dense), '\t', x.shape)
    return x

def downsample(x, num_dense):
    skip = dense_block(x, num_dense)
    skip = tf.concat([x, skip], axis=-1)
    x = transition_down(skip, num_dense*k + x.shape[-1].value)
    print('DB ({} layers) + TD'.format(num_dense), '\t', x.shape)
    return x, skip

with tf.name_scope('tiramisu'):
    # DOWN SAMPLING
    x = convolution2d(x_pl, num_outputs=k, kernel_size=(3, 3),
                             stride=1, scope="pre-convolution")
    print('pre_conv', '\t\t', x.shape)

    skip = []
    for num_layers in layers_architecture:
        x, skipTmp = downsample(x, num_layers)
        skip.append(skipTmp)
                    
    # BOTTLENECK
    x = dense_block(x, layers_bottleneck)
    bottleneck_ext = x
    print('Bottleneck ({} layers)'.format(layers_bottleneck), '\t', x.shape)

    # UPSAMPLING
    for index in range(len(layers_architecture)-1, -1, -1):
        x = upsample(x, skip[index], layers_architecture[index], skip_up = index==0)

    upsampl_ext = x
    # Output layers
    x = convolution2d(x, num_outputs=num_classes, kernel_size=(1, 1),
                             stride=1, scope="post-convolution")
    
    post_conv = x
    print('post-convolution', '\t', x.shape)
    y = softmax(x)
    print('SoftMax output', '\t\t', y.shape)

print("Model built")

In [ ]:
with tf.variable_scope('loss'):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(y_pl * tf.log(y+1e-8), reduction_indices=[1])

    # averaging over samples
    cross_entropy = tf.reduce_mean(cross_entropy)

    
with tf.variable_scope('training'):
    # defining our optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

    # applying the gradients
    train_op = optimizer.minimize(cross_entropy)

    
with tf.variable_scope('performance'):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = tf.equal(tf.argmax(y, axis=-1), tf.argmax(y_pl, axis=-1))
    
    # averaging the one-hot encoded vector
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
tf.summary.scalar('Evaluation/loss', cross_entropy)
tf.summary.scalar('Evaluation/accuracy', accuracy)

# Memory limitation
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)

## Testing the forward path

In [ ]:
#Test the forward pass
x_batch, y_batch = next(dummy_batch_gen.gen_train())

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    y_pred = sess.run(fetches=y, feed_dict={x_pl: x_batch, is_training_pl: True})

assert y_pred.shape == y_batch.shape, "ERROR the output shape is not as expected!" \
        + " Output shape should be " + str(y_batch.shape) + ' but was ' + str(y_pred.shape)

print('Forward pass successful!')

## Training the model

In [ ]:
#Training Loop
batch_size = 10
max_epochs = 10
ITERATIONS = 1e4
seed = 42
LOG_FREQ = 100
VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for valition

batch_gen = data_utils.batch_generator(data, batch_size=batch_size, num_classes=num_classes,
                            num_iterations=max_epochs, seed=seed, val_size=VALIDATION_SIZE)

valid_loss, valid_accuracy = [], []
train_loss, train_accuracy = [], []
test_loss, test_accuracy = [], []

# setup and write summaries
timestr = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
logdir = os.path.join('logs', timestr)
summaries = tf.summary.merge_all()

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    summary_writer = tf.summary.FileWriter(os.path.split(logdir)[0], graph=sess.graph)
    
    summarywriter_train = tf.summary.FileWriter(os.path.join(logdir, 'train'), sess.graph)
    summarywriter_valid = tf.summary.FileWriter(os.path.join(logdir, 'valid'), sess.graph)
    
    sess.run(tf.global_variables_initializer())
    print('Begin training loop')

    j = 0
    try:
        for num, batch_train in enumerate(batch_gen.gen_train()):
            if num>=ITERATIONS:
                break
            _train_loss, _train_accuracy = [], []
            
            print("epoch", num, j)
            j += 1
            ## Run train op
            x_batch = batch_train[0]
            y_batch = batch_train[1]
            fetches_train = [train_op, cross_entropy, accuracy, summaries, y, bottleneck_ext, upsampl_ext, post_conv]
            feed_dict_train = {x_pl: x_batch, y_pl: y_batch, is_training_pl: True}
            _, _loss, _acc, sum_train, output_t, bot_out, up_out, pconv_out = sess.run(fetches_train, feed_dict_train)
            
            _train_loss.append(_loss)
            _train_accuracy.append(_acc)
            
            if True:
                plt.subplot(161)
                plt.imshow(x_batch[0].reshape((height, width)), cmap='gray')
                plt.subplot(162)
                plt.imshow(bot_out[0, :, :, 0], cmap='gray')
                plt.subplot(163)
                plt.imshow(up_out[0, :, :, 0], cmap='gray')
                plt.subplot(164)
                plt.imshow(pconv_out[0, :, :, 0], cmap='gray')
                plt.subplot(165)
                plt.imshow(output_t[0, :, :, 0].reshape((height, width)), cmap='gray')
                plt.show()
            
            if j % LOG_FREQ == 0:
                summarywriter_train.add_summary(sum_train, j) # save the train summary
                print('\nLog!\n')
            

            ## Compute validation loss and accuracy
            if num % 1 == 0: #\
                    #and mnist_data.train._index_in_epoch <= batch_size:
                train_loss.append(np.mean(_train_loss))
                train_accuracy.append(np.mean(_train_accuracy))
                cur_acc = 0
                cur_loss = 0
                tot_num = 0
                # batch validation
                for numval, x_valid, y_valid in batch_gen.gen_valid():
                    fetches_valid = [cross_entropy, accuracy, summaries, y]
                    feed_dict_valid = {x_pl: x_valid, y_pl: y_valid, is_training_pl: False}
                    _loss, _acc, sum_valid, output_v = sess.run(fetches_valid, feed_dict_valid)

                    summarywriter_valid.add_summary(sum_valid, i) # save the valid summary
                    cur_acc += _acc*num
                    cur_loss += _loss*num
                    tot_num += num
                    
                valid_loss.append(cur_loss / float(tot_num))
                valid_accuracy.append((cur_acc / float(tot_num)))
                print("Epoch {} : Train Loss {:6.3f}, Train acc {:6.3f},  Valid loss {:6.3f},  Valid acc {:6.3f}".format(
                    num, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1]))
            

    except KeyboardInterrupt:
        pass